# K-Fold CV for Boosting Methods Using SKLearn 

In [2]:
#Importing required libraries
from sklearn.datasets import load_breast_cancer
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from tqdm import tqdm
import wfdb

from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from numpy import mean
from numpy import absolute
from numpy import sqrt
import pandas as pd
import random

In [3]:
import xgboost as xgb
from sklearn import metrics
from lightgbm import LGBMClassifier

import lightgbm
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [4]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.dpi'] = 70 #display 70 dpi in Jupyter Notebook, may consider100 dpi 
plt.rcParams['savefig.dpi'] = 300 #define 300 dpi for saving figures

import seaborn as sns
## here are some settings 
sns.set_style('whitegrid')
sns.set(rc={"figure.dpi":70, 'savefig.dpi':300}) #defining dpi setting
sns.set_context('notebook')
sns.set_style("ticks")
from IPython.display import set_matplotlib_formats

### To load sklearn dataset
sklearn.datasets.load_breast_cancer(*, return_X_y=False, as_frame=False)[source]¶

Load and return the breast cancer wisconsin dataset (classification).

In [5]:
#data = load_breast_cancer(as_frame = True); 
#df = data.frame; 
#print(df.shape)
# In this block use the code we looked at on 6/30 as a reference to save our rr-intervals as a .
rlist = []
records = 'mit-bih-raw\RECORDS' # Replace the _______ with the name of the records file in your MIT-BIH data folder
with open(records) as rfile: #Then we open the file 
                             #The 'with' command only opens the file while we are in it. Automatically closes the file when we're not
    for record in rfile:  # Then we iterate through the lines in the file
        record = record[0:len(record)-1] # Remove any erronious new line characters at the end ('\n')
        rlist.append(record) # Then build an array with it
        

###### Step 1: Initialize all Arrays
             # Below, replace all of the ___ with the command that declares an array/list
             # hint: https://stackoverflow.com/questions/1514553/how-to-declare-an-array-in-python
samples = [] # will house the samples of all subjects
good_list = [] # will list the names of the subjects we successfully extracted
bad_list = [] # will house the names of the subjects we failed to extract
qrs = [] # will house the indices of R-Peaks for all subjects
atr_label = [] # will house the labels for each rhythm annotation for all subjects
atr_locs = [] # will house the locations corresponding to the rhythm annotation labels


###### Step 2: Extract Information
for x in tqdm(rlist): #this will iterate through te records that we found above
  
    try: # A try statement will run the except statement if for some reason the try commands fail
         # In this case I use the try statement because one of the subjects has no signal data causing failure
         # I then use bad_list and good_list so that all of the indices in rlist match with the arrays we initialized in Step 1, above
        ######################################################
            # Below find the wfdb function that will return the information that is described below 
        # Then replace _____ with the correct function call
        samp = wfdb.rdsamp('mit-bih-raw/'+x) # wfdb._____(file_location) will read the signal & header data and return a 2 value array
            # samp[0] - the signal data is the raw reading from the ecg. Each value is a sample taken.
            # samp[1] - the header data includes things about the signal data such as:
              # samples per section, denoted 'fs'
              # number of signals, denoted 'n_sig'
            
        ######################################################
        samples.append(samp) #add it to our array for all subject
            #What is our file extension that has the annotation we want? Find it here and replace _____ with it 
            #hint: READ THE VARIABLE NAMES!!!!
        qrs_tmp = wfdb.rdann('mit-bih-raw/'+x, extension='qrs') #extract the QRS Info
        qrs_locs = np.array(qrs_tmp.sample, dtype='int') #Get just the loccation of R-Peaks from the QRS Info
        qrs.append(qrs_locs) # Add to our array for all subjects
        
            #Do the same thing here
        atr = wfdb.rdann('mit-bih-raw/'+x,extension='atr') #extract the atr info which stores the rhythm type(s) over the whole signal
        atr_label.append(atr.aux_note) # aux_note stores the type of rhythm - main two are '(N' for normal and '(AFIB' for AFIB
        atr_locs.append(np.append(atr.sample, len(samp[0]))) #I add the length of the whole sample to the end for better visualization later
        
        good_list.append(x) # when all extraction is successful append the record name to good_list
    except Exception as exep:
        print(exep) # Alert the user of an exception
        bad_list.append(x) # add to the bad list

        

  0%|          | 0/25 [00:00<?, ?it/s]

sampto must be greater than sampfrom
sampto must be greater than sampfrom


100%|██████████| 25/25 [00:21<00:00,  1.17it/s]


In [6]:
features_df_list = []
for subject in good_list:
    data = pd.read_csv(f'subject-features-dataframes/subject{subject}.csv')
    data = data.assign(Subject = subject)
    #print(data)
    features_df_list.append(data)
#features_maindf = pd.concat(features_df_list, ignore_index = True)

In [7]:
features_maindf = pd.concat(features_df_list, ignore_index = True)
print(features_maindf.columns)

Index(['Unnamed: 0', 'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
       'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
       'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation',
       'Rhythm', 'Subject'],
      dtype='object')


In [12]:
def confusion_calcs(confusion_matrix):
    temp_arr = confusion_matrix
    tn = temp_arr[0]
    fp = temp_arr[1]
    fn = temp_arr[2]
    tp = temp_arr[3]
    sensitivity = tp / (tp + fn)
    recall = sensitivity
    specificity = tn / (tn + fp)
    precision = tp / (tp + fp)
    accuracy = (tp+tn) / (tp+tn+fn+fp)
    f1 = 2*precision*recall/(precision+recall)
    return [accuracy, sensitivity, specificity, precision, f1]

In [13]:
# Short to Short

import lightgbm

#Implementing cross validation 
#kf = KFold(n_splits=23, random_state=1, shuffle=True)
model = lightgbm.LGBMClassifier(learning_rate=0.1, n_estimators=100, max_depth=5, random_state=0)

from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

y_values = ['Rhythm']
df = features_maindf



all_features = ['StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
        'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
        'AppEn', 'SampEn']
dataScores = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
        
for score_idx,temp in tqdm(enumerate(all_features)):    
    
    x_values = [temp]
    print(x_values)

    acc_score = [];
    Truth = [];
    Output = [];


    for idx, subject in enumerate(good_list):

        train_index = ~df['Subject'].str.contains(subject)
        test_index = df['Subject'].str.contains(subject)
        X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
        y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values]

        # Create the LightGBM data containers
        model.fit(X_train,y_train,eval_set=[(X_test,y_test),(X_train,y_train)],
              eval_metric='logloss')

        pred_values = model.predict(X_test)

        acc = accuracy_score(y_test, pred_values)
        acc_score.append(acc)

        Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
        Output.extend(pred_values); ## it is a list  

        #print(model.feature_importances_)
    #     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
    #                           index = (X_train.columns))
    #     Importance.sort_values(by = 'Importance', 
    #                        axis = 0, 
    #                        ascending = True).plot(kind = 'barh', 
    #                                               color = 'r')
    #     plt.xlabel('Variable Importance')
    #     plt.gca().legend_ = None
        #plt.savefig('plot1.png')


    elapsed = timeit.default_timer() - start_time #gives total computation time
    print("---Run time is %s seconds ---" % elapsed) #prints computation time
    print()

    print('Accuracy of each fold: \n {}'.format(acc_score))
    print()
    print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
    print()
    print('Std of accuracy : \n{}'.format(np.std(acc_score)))

    print(confusion_matrix(Truth, Output))
    print(classification_report(Truth, Output))
    print(acc_score)
    print(elapsed)
    
    temp_array = confusion_matrix(Truth, Output)
    joint = temp_array[0].tolist() + temp_array[1].tolist()
    temp_scores = confusion_calcs(joint)
    
    #dataScores[score_idx].append(all_features[score_idx])
    #dataScores[score_idx].append(acc_score)
    dataScores[score_idx].append(np.mean(acc_score))
    dataScores[score_idx].append(np.std(acc_score))
    
    for confusionScore in temp_scores:
        dataScores[score_idx].append(confusionScore)

0it [00:00, ?it/s]C:\Users\NNL\anacondareal\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


['StoS']
[1]	training's binary_logloss: 0.663169	valid_0's binary_logloss: 0.583485
[2]	training's binary_logloss: 0.640457	valid_0's binary_logloss: 0.543097
[3]	training's binary_logloss: 0.621773	valid_0's binary_logloss: 0.508882
[4]	training's binary_logloss: 0.606301	valid_0's binary_logloss: 0.479678
[5]	training's binary_logloss: 0.593425	valid_0's binary_logloss: 0.454601
[6]	training's binary_logloss: 0.58267	valid_0's binary_logloss: 0.432963
[7]	training's binary_logloss: 0.573665	valid_0's binary_logloss: 0.414217
[8]	training's binary_logloss: 0.566112	valid_0's binary_logloss: 0.397925
[9]	training's binary_logloss: 0.559769	valid_0's binary_logloss: 0.383726
[10]	training's binary_logloss: 0.55444	valid_0's binary_logloss: 0.371324
[11]	training's binary_logloss: 0.549961	valid_0's binary_logloss: 0.360472
[12]	training's binary_logloss: 0.546199	valid_0's binary_logloss: 0.350961
[13]	training's binary_logloss: 0.543039	valid_0's binary_logloss: 0.342617
[14]	training'

C:\Users\NNL\anacondareal\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\NNL\anacondareal\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[1]	training's binary_logloss: 0.657848	valid_0's binary_logloss: 0.669395
[2]	training's binary_logloss: 0.631386	valid_0's binary_logloss: 0.683584
[3]	training's binary_logloss: 0.609616	valid_0's binary_logloss: 0.697856
[4]	training's binary_logloss: 0.591586	valid_0's binary_logloss: 0.711926
[5]	training's binary_logloss: 0.576581	valid_0's binary_logloss: 0.725602
[6]	training's binary_logloss: 0.564049	valid_0's binary_logloss: 0.738756
[7]	training's binary_logloss: 0.553557	valid_0's binary_logloss: 0.751305
[8]	training's binary_logloss: 0.544757	valid_0's binary_logloss: 0.763198
[9]	training's binary_logloss: 0.537369	valid_0's binary_logloss: 0.774409
[10]	training's binary_logloss: 0.531162	valid_0's binary_logloss: 0.784925
[11]	training's binary_logloss: 0.525948	valid_0's binary_logloss: 0.794752
[12]	training's binary_logloss: 0.521568	valid_0's binary_logloss: 0.803903
[13]	training's binary_logloss: 0.517891	valid_0's binary_logloss: 0.812396
[14]	training's binar

C:\Users\NNL\anacondareal\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\NNL\anacondareal\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


...

[1]	training's binary_logloss: 0.622853	valid_0's binary_logloss: 0.564803
[2]	training's binary_logloss: 0.567494	valid_0's binary_logloss: 0.509159
[3]	training's binary_logloss: 0.52152	valid_0's binary_logloss: 0.462632
[4]	training's binary_logloss: 0.482898	valid_0's binary_logloss: 0.423345
[5]	training's binary_logloss: 0.450122	valid_0's binary_logloss: 0.389991
[6]	training's binary_logloss: 0.422086	valid_0's binary_logloss: 0.361262
[7]	training's binary_logloss: 0.397974	valid_0's binary_logloss: 0.336557
[8]	training's binary_logloss: 0.377103	valid_0's binary_logloss: 0.315111
[9]	training's binary_logloss: 0.358997	valid_0's binary_logloss: 0.296249
[10]	training's binary_logloss: 0.343237	valid_0's binary_logloss: 0.279783
[11]	training's binary_logloss: 0.32947	valid_0's binary_logloss: 0.26531
[12]	training's binary_logloss: 0.317414	valid_0's binary_logloss: 0.252695
[13]	training's binary_logloss: 0.306874	valid_0's binary_logloss: 0.241517
[14]	training's binary_l

C:\Users\NNL\anacondareal\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[1]	training's binary_logloss: 0.617279	valid_0's binary_logloss: 0.679347
[2]	training's binary_logloss: 0.562688	valid_0's binary_logloss: 0.608381
[3]	training's binary_logloss: 0.517421	valid_0's binary_logloss: 0.549444
[4]	training's binary_logloss: 0.479414	valid_0's binary_logloss: 0.499834
[5]	training's binary_logloss: 0.447195	valid_0's binary_logloss: 0.457648
[6]	training's binary_logloss: 0.419718	valid_0's binary_logloss: 0.421439
[7]	training's binary_logloss: 0.3961	valid_0's binary_logloss: 0.390149
[8]	training's binary_logloss: 0.375683	valid_0's binary_logloss: 0.362856
[9]	training's binary_logloss: 0.357943	valid_0's binary_logloss: 0.338922
[10]	training's binary_logloss: 0.342523	valid_0's binary_logloss: 0.317966
[11]	training's binary_logloss: 0.329056	valid_0's binary_logloss: 0.299451
[12]	training's binary_logloss: 0.317277	valid_0's binary_logloss: 0.28305
[13]	training's binary_logloss: 0.306973	valid_0's binary_logloss: 0.268571
[14]	training's binary_l

16it [01:33,  5.84s/it]

[[21079  3664]
 [ 2172 18053]]
              precision    recall  f1-score   support

           0       0.91      0.85      0.88     24743
           1       0.83      0.89      0.86     20225

    accuracy                           0.87     44968
   macro avg       0.87      0.87      0.87     44968
weighted avg       0.87      0.87      0.87     44968

[0.8850114416475973, 0.947261663286004, 0.9924433249370277, 0.6157074340527577, 0.9801462904911181, 0.9253549695740365, 0.9061186361513311, 0.988427501701838, 0.9272088353413654, 0.7932745314222712, 0.9818511796733213, 0.9576453697056713, 0.7034013605442176, 1.0, 0.3800999167360533, 0.8896765618077094, 0.9671007539410555, 0.9976918638199653, 0.5815722738799662, 0.9521715228147334, 0.9773310521813516, 0.9127432517263026, 0.9661733615221987]
93.21077309999998


In [14]:
export_df = pd.DataFrame(dataScores,columns=['Mean Acc.', 'SD of Acc.', 'Total Acc.', 'Sensitivity', 
                                             'Specificity','Precision','f1'])

export_df = export_df.set_index(pd.Index(['StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
        'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
        'AppEn', 'SampEn']))

In [15]:
export_df

,Mean Acc.,SD of Acc.,Total Acc.,Sensitivity,Specificity,Precision,f1
StoS,0.692705,0.266921,0.667052,0.726873,0.618155,0.608762,0.662595
StoR,0.898542,0.112898,0.891456,0.934487,0.856283,0.841646,0.885640
StoL,0.721097,0.159525,0.708993,0.583189,0.811826,0.716978,0.643200
RtoS,0.845295,0.132321,0.836217,0.841978,0.831508,0.803331,0.822200
RtoR,0.883605,0.139107,0.875200,0.901607,0.853615,0.834286,0.866641
RtoL,0.907903,0.077323,0.901530,0.859679,0.935739,0.916214,0.887047
LtoS,0.785057,0.178731,0.769147,0.542744,0.954209,0.906441,0.678955
LtoR,0.822828,0.118022,0.820428,0.856663,0.790810,0.769976,0.811009
LtoL,0.692136,0.243382,0.670432,0.306452,0.967951,0.886568,0.455467
STD,0.784919,0.194159,0.783335,0.867441,0.714586,0.712997,0.782673


In [16]:
export_df_highlight = export_df.copy()

export_df_highlight.style.highlight_max(color = 'lightgreen', axis = 0)

,Mean Acc.,SD of Acc.,Total Acc.,Sensitivity,Specificity,Precision,f1
StoS,0.692705,0.266921,0.667052,0.726873,0.618155,0.608762,0.662595
StoR,0.898542,0.112898,0.891456,0.934487,0.856283,0.841646,0.885640
StoL,0.721097,0.159525,0.708993,0.583189,0.811826,0.716978,0.643200
RtoS,0.845295,0.132321,0.836217,0.841978,0.831508,0.803331,0.822200
RtoR,0.883605,0.139107,0.875200,0.901607,0.853615,0.834286,0.866641
RtoL,0.907903,0.077323,0.901530,0.859679,0.935739,0.916214,0.887047
LtoS,0.785057,0.178731,0.769147,0.542744,0.954209,0.906441,0.678955
LtoR,0.822828,0.118022,0.820428,0.856663,0.790810,0.769976,0.811009
LtoL,0.692136,0.243382,0.670432,0.306452,0.967951,0.886568,0.455467
STD,0.784919,0.194159,0.783335,0.867441,0.714586,0.712997,0.782673


In [17]:
export_df_highlight.style.highlight_min(color = 'red', axis = 0)

,Mean Acc.,SD of Acc.,Total Acc.,Sensitivity,Specificity,Precision,f1
StoS,0.692705,0.266921,0.667052,0.726873,0.618155,0.608762,0.662595
StoR,0.898542,0.112898,0.891456,0.934487,0.856283,0.841646,0.885640
StoL,0.721097,0.159525,0.708993,0.583189,0.811826,0.716978,0.643200
RtoS,0.845295,0.132321,0.836217,0.841978,0.831508,0.803331,0.822200
RtoR,0.883605,0.139107,0.875200,0.901607,0.853615,0.834286,0.866641
RtoL,0.907903,0.077323,0.901530,0.859679,0.935739,0.916214,0.887047
LtoS,0.785057,0.178731,0.769147,0.542744,0.954209,0.906441,0.678955
LtoR,0.822828,0.118022,0.820428,0.856663,0.790810,0.769976,0.811009
LtoL,0.692136,0.243382,0.670432,0.306452,0.967951,0.886568,0.455467
STD,0.784919,0.194159,0.783335,0.867441,0.714586,0.712997,0.782673


In [18]:
export_df.to_csv(f'individual-features/LightGBM.csv')